In [1]:
import os
import json
import warnings
import functools

import glob as gb
import utils as ut
import numpy as np
import pandas as pd
import seaborn as sns
import ipywidgets as pyw
import matplotlib.pyplot as plt

from tqdm import tqdm


In [2]:
%matplotlib inline


# Global

In [3]:
dark_mode = False


In [4]:
warnings.filterwarnings('ignore')
plt.rcParams['text.usetex'] = True
plt.rcParams['figure.max_open_warning'] = 100
plt.style.use(['dark_background' if dark_mode else 'default'])
pd.set_option('display.max_colwidth', None)


# Data

## Load

In [5]:
# init data
df, data = pd.DataFrame(), {}

# load pre-processed data
root = ut.get_value(json.load(open('settings.json')), 'data.output')
for path_data in tqdm(sorted(gb.glob(os.path.join(root, 'forest-21', '**', 'data*.npz'), recursive=True))):

    # file directory and file name
    file_directory = os.path.dirname(path_data)
    file_name, _ = os.path.splitext(os.path.basename(path_data))

    # load simulation parameters
    parameters_json = next(iter(gb.glob(os.path.join(file_directory, 'parameters*.json'))), None)
    if not parameters_json:
        continue
    parameters = json.load(open(parameters_json))
    ut.del_keys(parameters, 'forest.persons')
    ut.del_keys(parameters, 'material')
    ut.del_keys(parameters, 'capture')
    ut.del_keys(parameters, 'next')
    ut.del_keys(parameters, 'url')

    # get parameter values
    preset = ut.get_value(parameters, 'preset')
    size = ut.get_value(parameters, 'forest.size')
    height = ut.get_value(parameters, 'drone.height')
    coverage = ut.get_value(parameters, 'drone.coverage')
    view = ut.get_value(parameters, 'drone.camera.view')
    sampling = ut.get_value(parameters, 'drone.camera.sampling')

    # hardcoded number/distance of captures from file name
    N, M = ut.sample_data(parameters)
    n = int(file_name.split('-')[-1][1:])
    m = M[n]

    # hardcoded perspective/orthographic camera from file name
    c = 'O' if 'orthographic' in file_directory else 'P'

    # simulation name, group and ground image
    simulation = os.path.basename(file_directory)
    folder = os.path.basename(os.path.dirname(file_directory))
    prefixname = 'forest simplified' if 'cone' in folder else 'forest'
    suffixname = 'sparse' if size <= 300 else ('dense' if size >= 900 else 'medium')
    displayname = f'{prefixname} {suffixname}'
    subgroup = os.path.basename(os.path.dirname(file_directory))
    group = os.path.basename(os.path.dirname(os.path.dirname(file_directory)))
    name = f'{preset}-F{size:04}-N{n:03}-H{height}-V{view}-S{sampling:.1f}-{c}'
    image = gb.glob(os.path.join(file_directory, f'ground-*-N{n:03d}.png'))[0]

    # load data
    data[simulation] = np.load(path_data, allow_pickle=True)
    if 'statistics' in data[simulation]:
        statistics = data[simulation]['statistics'].item()
    else:
        statistics = {'trees_per_image': -1, 'ground_visibility': data[simulation]['visibility'].item()}

    # statistics data
    trees = statistics['trees_per_image']
    visibility = statistics['ground_visibility']
    density = 1.0 - visibility

    # append simulation data
    df = df.append(pd.json_normalize({
        'name': name,
        'displayname': displayname,
        'group': group,
        'subgroup': subgroup,
        'simulation': simulation,
        'visibility': visibility,
        'density': density,
        'trees': trees,
        'image': image,
        'C': c,
        'N': n,
        'M': m,
        **parameters
    }), ignore_index=True)


100%|██████████| 95/95 [00:00<00:00, 118.05it/s]


## Preview

In [6]:
# sort and preview data
df = df.sort_values(['group', 'subgroup', 'drone.camera.view'], ascending=True)
df = df.reset_index(drop=True)
df = df.fillna(0.0)
df


,name,displayname,group,subgroup,simulation,visibility,density,trees,image,C,...,drone.speed,drone.height,drone.rotation,drone.camera.view,drone.camera.resolution,drone.camera.sampling,drone.camera.type,drone.coverage,forest.size,forest.ground
0,forest-21-F0300-N001-H40-V20-S1.0-P,forest sparse,output,forest-21,AOS-Simulation-2022-03-06-17-41-14,0.351956,0.648044,3.360000,./data/output/forest-21/AOS-Simulation-2022-03-06-17-41-14/ground-forest-21-N001.png,P,...,10,40,0,20,512,1,monochrome,14.106158,300,70
1,forest-21-F0300-N002-H40-V20-S1.0-P,forest sparse,output,forest-21,AOS-Simulation-2022-03-06-17-41-14,0.497723,0.502277,3.300000,./data/output/forest-21/AOS-Simulation-2022-03-06-17-41-14/ground-forest-21-N002.png,P,...,10,40,0,20,512,1,monochrome,14.106158,300,70
2,forest-21-F0300-N003-H40-V20-S1.0-P,forest sparse,output,forest-21,AOS-Simulation-2022-03-06-17-41-14,0.555845,0.444155,3.314286,./data/output/forest-21/AOS-Simulation-2022-03-06-17-41-14/ground-forest-21-N003.png,P,...,10,40,0,20,512,1,monochrome,14.106158,300,70
3,forest-21-F0300-N004-H40-V20-S1.0-P,forest sparse,output,forest-21,AOS-Simulation-2022-03-06-17-41-14,0.590683,0.409317,3.284091,./data/output/forest-21/AOS-Simulation-2022-03-06-17-41-14/ground-forest-21-N004.png,P,...,10,40,0,20,512,1,monochrome,14.106158,300,70
4,forest-21-F0300-N005-H40-V20-S1.0-P,forest sparse,output,forest-21,AOS-Simulation-2022-03-06-17-41-14,0.632874,0.367126,3.267241,./data/output/forest-21/AOS-Simulation-2022-03-06-17-41-14/ground-forest-21-N005.png,P,...,10,40,0,20,512,1,monochrome,14.106158,300,70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,forest-21-F0300-N016-H40-V90-S1.0-P,forest sparse,output,forest-21,AOS-Simulation-2022-03-06-17-37-41,0.768939,0.231061,99.785714,./data/output/forest-21/AOS-Simulation-2022-03-06-17-37-41/ground-forest-21-N016.png,P,...,10,40,0,90,512,1,monochrome,80.000000,300,70
91,forest-21-F0300-N020-H40-V90-S1.0-P,forest sparse,output,forest-21,AOS-Simulation-2022-03-06-17-37-41,0.801836,0.198164,100.235294,./data/output/forest-21/AOS-Simulation-2022-03-06-17-37-41/ground-forest-21-N020.png,P,...,10,40,0,90,512,1,monochrome,80.000000,300,70
92,forest-21-F0300-N027-H40-V90-S1.0-P,forest sparse,output,forest-21,AOS-Simulation-2022-03-06-17-37-41,0.835383,0.164617,100.173913,./data/output/forest-21/AOS-Simulation-2022-03-06-17-37-41/ground-forest-21-N027.png,P,...,10,40,0,90,512,1,monochrome,80.000000,300,70
93,forest-21-F0300-N040-H40-V90-S1.0-P,forest sparse,output,forest-21,AOS-Simulation-2022-03-06-17-37-41,0.871697,0.128303,99.942857,./data/output/forest-21/AOS-Simulation-2022-03-06-17-37-41/ground-forest-21-N040.png,P,...,10,40,0,90,512,1,monochrome,80.000000,300,70


In [7]:
# extract unique subgroups
subgroups = sorted(df['subgroup'].unique(), key=lambda x: x[10:])
subgroups


['forest-21']

## Helper

In [8]:
def selection_mask(df, **kwargs):
    masks = [df[k].isin(v) for k, v in kwargs.items()]
    return functools.reduce(np.logical_and, masks)


In [9]:
def sampling_mask(df, **kwargs):
    G = df['subgroup'].unique()
    V = df['drone.camera.view'].unique()
    views = {g: {v: [] for v in V} for g in G}

    # step size
    key, value = next(((k, v) for k, v in kwargs.items()))
    steps = {
        'N': np.arange(0, 10, 1),
        'M': np.round(np.arange(0.0, 2.0, 0.1), 2)
    }[key]

    # estimate nearest N/M for data unavailable
    for subgroup in df['subgroup'].unique():
        for step in steps:
            subgroup_mask = (df['subgroup'] == subgroup) & np.isclose(df[key], value + step)
            for idx, view in df[subgroup_mask]['drone.camera.view'].items():
                views[subgroup][view].append(idx)

    # use first obtained indices
    idxs = np.array([[g[0] for g in v.values() if len(g)] for v in views.values()]).flatten()

    # return mask
    return df.index.isin(idxs)


# Plots

## Visibility

In [10]:
def plot_visibilities_1(df, group, title, file, ncol, **kwargs):

    # group dataframe
    group_name_0, group_name_1 = group
    grouped_0 = df.groupby(group_name_0, sort=False)

    # aggregate data
    for i, (name_0, group_0) in enumerate(grouped_0):

        # initialize subplots
        fig, ax = plt.subplots(figsize=(8, 6))
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)

        # initialize styles
        grouped_1 = group_0.groupby(group_name_1, sort=False)
        colors = [x for i, x in enumerate(sns.color_palette('deep')) if i not in [7]]
        markers = {
            0.5: ('P', 30),
            1.0: ('^', 25),
            1.5: ('s', 20),
            2.0: ('o', 25)
        }
        linestyles = {
            300: 'dotted',
            600: 'dashed',
            900: 'solid'
        }

        # plot visibilities
        ps = []
        samplings = []
        for j, (name_1, group_1) in enumerate(grouped_1):

            N = group_1['N'].iloc[j]
            view = group_1['drone.camera.view'].iloc[j]
            height = group_1['drone.height'].iloc[j]
            coverage = group_1['drone.coverage'].iloc[j]
            size = group_1['forest.size'].iloc[j]

            # styles
            c = {
                20: colors[0],
                30: colors[1],
                40: colors[2],
                50: colors[3],
                60: colors[4],
                70: colors[5],
                80: colors[6],
                90: colors[7]
            }
            color = c[view]
            linestyle = linestyles[size]

            # sampling points
            sampling = []
            for M, marker in markers.items():
                df_sampling = group_1[sampling_mask(group_1, M=M)]
                X, Y = df_sampling['N'], df_sampling['visibility']
                if j == 0:
                    p = ax.scatter(X, Y, marker=marker[0], s=marker[1], color='#666666', zorder=2)
                    ps.append([p, f'sampling (d={M})'])
                p = ax.scatter(X, Y, marker=marker[0], s=marker[1], color=color, zorder=2)
                ps.append([p, f'sampling (d={M})'])
                sampling.append([X.item() if X.size else 0, Y.item() if Y.size else 0])
            samplings.append(sampling)

            # simulation visibilities
            group_1.sort_values(by='N', inplace=True)
            group_1.plot(kind='line', x='N', y='visibility', label=name_1, linestyle='solid', color=color, zorder=1, ax=ax, legend=False)

            # plot title
            formats = {
                'displayname': '{0}',
                'drone.height': '{0}m height',
                'drone.camera.view': 'field of view {0}°'
            }
            ax.set_title(formats[title].format(str(group_1[title].iloc[j])))

        # add first artist legend
        ps = np.array(ps)
        txt, idx = np.unique(ps[:, 1], return_index=True)
        legend = ax.legend(ps[idx, 0], txt, loc='best')
        ax.add_artist(legend)

        # sampling point lines
        if title == 'displayname':
            colors = [x for x in ut.colors(cmap='Paired', size=3)] * 3
            colors = {
                30: colors[0],
                35: colors[0],
                40: colors[1],
                50: colors[2],
                27: colors[1],
                41: colors[2]

            }
            samplings = np.stack(samplings).T
            for X, Y in zip(samplings[0], samplings[1]):
                ax.plot(X[X != 0], Y[Y != 0], color=colors[height], linestyle=linestyles[size], zorder=0)

        # set axis
        ax.set_xlim(np.add(kwargs['N'], [0, 0]))
        ax.set_ylim([0, 1.0])

        # set labels
        ax.set_xlabel(r'number of images $n$')
        ax.set_ylabel(r'visibility $\dot{V}$')

        # set legend
        handles, labels = ax.get_legend_handles_labels()
        labels, handles = zip(*sorted(zip(labels, handles), key=lambda x: x[0], reverse=False))
        ax.legend(handles, labels, loc='upper center', bbox_to_anchor=(0.5, -0.12), ncol=ncol, fancybox=True, shadow=False)

        # export
        ut.export_plot(fig, os.path.join('results', 'visibilities', f'{file}-{name_0}.png'), False)


In [11]:
def plot_visibilities_2(df, group, title, file, ncol, **kwargs):

    # group dataframe
    group_name_0, group_name_1 = group
    grouped_0 = df.groupby(group_name_0)

    # initialize subplots
    fig, ax = plt.subplots(figsize=(7, 5))
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

    # initialize styles
    colors = [x for x in ut.colors(cmap='Paired', size=3)] * 3
    markers = {
        30: ('x', 30),
        35: ('x', 30),
        40: ('x', 30),
        50: ('x', 30),
        27: ('x', 30),
        41: ('x', 30)
    }
    linestyles = {
        300: 'dotted',
        600: 'dashed',
        900: 'solid'
    }

    # plot visibilities
    ps = []
    for i, (name_0, group_0) in enumerate(grouped_0):
        name = group_0[group_name_1].iloc[0]
        size = group_0['forest.size'].iloc[0]
        height = group_0['drone.height'].iloc[0]

        # styles
        color = colors[i]
        marker = markers[height]
        linestyle = linestyles[size]

        # extrema points
        i = group_0['visibility'].argmax()
        x = group_0['drone.camera.view'].iloc[i]
        y = group_0['visibility'].iloc[i]
        p = ax.scatter(x, y, marker=marker[0], s=marker[1], color=color, zorder=3)
        ps.append([p, f'maximum (h={height})'])

        # simulation visibilities
        group_0.sort_values(by='drone.camera.view', inplace=True)
        group_0.plot(kind='line', x='drone.camera.view', y='visibility', label=name, linestyle=linestyle, color=color, ax=ax, legend=False)

    # add first artist legend
    ps = np.array(ps)
    txt, idx = np.unique(ps[:, 1], return_index=True)
    legend = ax.legend(ps[idx, 0], txt, loc='best')
    ax.add_artist(legend)

    # plot title
    formats = {
        'M': 'sampling distance {0:.1f}m',
        'N': 'number of images {0}'
    }
    ax.set_title(formats[title].format(kwargs[title]))

    # set axis
    ax.set_xlim(np.add([20, 90], [0, 5]))
    ax.set_ylim([0, 1.0])

    # set labels
    ax.set_xlabel(r'field of view [°]')
    ax.set_ylabel(r'visibility $\dot{V}$')

    # set legend
    handles, labels = ax.get_legend_handles_labels()
    labels, handles = zip(*sorted(zip(labels, handles), key=lambda x: len(x[0]), reverse=True))
    ax.legend(handles, labels, loc='upper center', bbox_to_anchor=(0.5, -0.12), ncol=ncol, fancybox=True, shadow=False)

    # export
    formats = {
        'M': 'sampling-distance-{0:.1f}',
        'N': 'number-of-images-{0}'
    }
    name = formats[title].format(kwargs[title])
    ut.export_plot(fig, os.path.join('results', 'visibilities', f'{file}-{name}.png'), False)


### Field of view

In [12]:
@pyw.interact
def plot(G=pyw.SelectMultiple(value=subgroups, options=subgroups, continuous_update=False),
         N=pyw.IntRangeSlider(value=(1, 105), min=1, max=400, continuous_update=False)):

    # mask
    mask = selection_mask(df, subgroup=G)
    df_masked = df[mask].copy()

    # values
    view = df_masked['drone.camera.view'].astype('str')
    height = 'h=' + np.round(df_masked['drone.height'], 1).astype('str') + 'm'
    coverage = 'c=' + np.round(df_masked['drone.coverage'], 1).astype('str') + 'm'

    # group
    df_masked['plotgroup.0'] = df_masked['subgroup']
    df_masked['plotgroup.1'] = 'FOV=' + view + '° (' + height + ', ' + coverage + ')'

    # plot
    plot_visibilities_1(df_masked, ['plotgroup.0', 'plotgroup.1'], 'displayname', 'plot-1-1', 2, G=G, N=N)


interactive(children=(SelectMultiple(description='G', index=(0,), options=('forest-21',), value=('forest-21',)…

### Sampling distance

In [13]:
@pyw.interact
def plot(G=pyw.SelectMultiple(value=subgroups, options=subgroups, continuous_update=False),
         M=pyw.FloatSlider(value=1.0, min=0.1, max=2.0, step=0.1, continuous_update=True)):

    # mask
    mask = selection_mask(df, subgroup=G) & sampling_mask(df, M=M)
    df_masked = df[mask].copy()

    # values
    height = 'h=' + np.round(df_masked['drone.height'], 1).astype('str') + 'm'

    # group
    df_masked['plotgroup.0'] = df_masked['subgroup']
    df_masked['plotgroup.1'] = df_masked['displayname'] + ' (' + height + ')'

    # plot
    plot_visibilities_2(df_masked, ['plotgroup.0', 'plotgroup.1'], 'M', 'plot-2-1', 3, G=G, M=M)


interactive(children=(SelectMultiple(description='G', index=(0,), options=('forest-21',), value=('forest-21',)…

### Number of images

In [14]:
@pyw.interact
def plot(G=pyw.SelectMultiple(value=subgroups, options=subgroups, continuous_update=False),
         N=pyw.IntSlider(value=1, min=1, max=35, step=1, continuous_update=True)):

    # mask
    mask = selection_mask(df, subgroup=G) & sampling_mask(df, N=N)
    df_masked = df[mask].copy()

    # values
    height = 'h=' + np.round(df_masked['drone.height'], 1).astype('str') + 'm'

    # group
    df_masked['plotgroup.0'] = df_masked['subgroup']
    df_masked['plotgroup.1'] = df_masked['displayname'] + ' (' + height + ')'

    # plot
    plot_visibilities_2(df_masked, ['plotgroup.0', 'plotgroup.1'], 'N', 'plot-2-2', 3, G=G, N=N)


interactive(children=(SelectMultiple(description='G', index=(0,), options=('forest-21',), value=('forest-21',)…